Since recently I've started some experimenting with cryptocurrency. So I decided to get my own parser for crypto.
Started from Binance, hopefully will get more sites to this.

In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

Don't blame me if I'm wrong in some things, this is my first experience with websites and selenium library.
If you have some improvement ideas, welcome, I'l be really greatful!

Before coming to selenium library, I've studied BeautifulSoap. Last one appeared to be not effective since Binance has dinamic nature.

In [2]:
url = 'https://www.binance.com/ru/markets'

In [3]:
driver = webdriver.Chrome('C:/Users/under/Desktop/chromedriver.exe')
driver.get(url)

After starting browser next cell will click on 'Zones' button in order to get us to cryptos and its cost list

In [4]:
zones_button = driver.find_element_by_xpath('//*[@id="__APP"]/div[1]/main/div/div[2]/div/div[2]/div[1]/div/div/div/div[2]/div')
ActionChains(driver).click(zones_button).perform()

Next cells own some explanation. Main values - crypto name and crypto cost - lie in next 2 paths.
The problem is that after scrolling down on the page, scrolled up and lying downside values(which are not visible to user) are dissapearing from DOM structure.
Meaning that user can see up to 16 values and only them will exist in DOM.
After scrolling, new ones will replace previous and will never reach more than 16 count(in this place in Xpath - "/div[]/")
So script goes through all visible and existing in DOM values, writes all of them in crypto_list_2, and scrolling down to get new ones.

In [5]:
path_1 = '//*[@id="__APP"]/div[1]/main/div/div[2]/div/div[2]/div[2]/div[2]/div/div[2]/div/div/div[]/div/div[3]/div'
path_0 = '//*[@id="__APP"]/div[1]/main/div/div[2]/div/div[2]/div[2]/div[2]/div/div[2]/div/div/div[]/div/div[2]/div[2]/div[1]'

In [6]:
import time 

In [7]:
crypto_list = []

In [8]:
%%time

elm = driver.find_element_by_tag_name("html")

start_time = time.time()

for i in range(1, 100):
    print('Iteration count:', i)
    cryptos = driver.find_elements_by_class_name('css-4cffwv')
    for k in range(1, 16):
        title_path = path_0[:88] + str(k) + path_0[88:]
        price_path = path_1[:88] + str(k) + path_1[88:]
        for crypto in cryptos:
            title = crypto.find_element_by_xpath(title_path).text
            price = crypto.find_element_by_xpath(price_path).text
            
            crypto_list_dict = {'crypto':title,
                                'price':price}
            crypto_list.append(crypto_list_dict)
            
            break
    for j in range(3):  
        elm.send_keys(Keys.DOWN)
        elm.send_keys(Keys.DOWN)
        elm.send_keys(Keys.DOWN)
        elm.send_keys(Keys.DOWN)
        time.sleep(1)
    print("--- %s seconds ---" % (time.time() - start_time))

Iteration count: 1
--- 4.348567962646484 seconds ---
Iteration count: 2
--- 9.82928204536438 seconds ---
Iteration count: 3
--- 15.231282949447632 seconds ---
Iteration count: 4
--- 20.477298736572266 seconds ---
Iteration count: 5
--- 25.969638347625732 seconds ---
Iteration count: 6
--- 31.842546463012695 seconds ---
Iteration count: 7
--- 37.2566614151001 seconds ---
Iteration count: 8
--- 42.98860239982605 seconds ---
Iteration count: 9
--- 48.44950819015503 seconds ---
Iteration count: 10
--- 53.96735644340515 seconds ---
Iteration count: 11
--- 59.473928928375244 seconds ---
Iteration count: 12
--- 65.40515732765198 seconds ---
Iteration count: 13
--- 70.84210991859436 seconds ---
Iteration count: 14
--- 76.29302763938904 seconds ---
Iteration count: 15
--- 81.56091117858887 seconds ---
Iteration count: 16
--- 87.20927238464355 seconds ---
Iteration count: 17
--- 92.67594146728516 seconds ---
Iteration count: 18
--- 98.10774970054626 seconds ---
Iteration count: 19
--- 103.316406

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="__APP"]/div[1]/main/div/div[2]/div/div[2]/div[2]/div[2]/div/div[2]/div/div/div[11]/div/div[2]/div[2]/div[1]"}
  (Session info: chrome=91.0.4472.124)


Script is ending with error but it has done its work and we've got data we need. I'll try to fix this in future edits.

In [9]:
crypto_list = pd.DataFrame(crypto_list)

In [10]:
crypto_list = crypto_list.drop_duplicates(subset='crypto', keep='first', inplace=False)

In [11]:
crypto_list

,crypto,price
0,BTC,"₽2,450,745.58"
1,ETH,"₽156,515.19"
2,USDT,₽74.32
3,BNB,"₽23,036.35"
4,ADA,₽99.00
...,...,...
717,XTZDOWN,₽0.150147
718,XTZUP,₽2.60
719,YFIDOWN,₽196.47
728,YFIUP,₽31.37


Original crypto_list_2 before dropping duplicates has 720 rows, which is nearly twice. I think it's inevitable to have duplicates but they surely can be lowered.
Duplicates exist because in this script we need slow scrolling to give a chance for cells on website to appear on screen and in DOM. 

-----------------------------------------------------------------------------------------------------------------------------

In [12]:
crypto_previous_day = pd.read_excel('C:/Users/under/Desktop/crypto_excels/crypto.xlsx')

In [13]:
crypto_previous_day = crypto_previous_day.merge(crypto_list, how='inner', on='crypto')

In [14]:
crypto_previous_day

,Unnamed: 0,Unnamed: 0.1,crypto,price_07/07/2021,price_08/07/2021,dinamic_08/07/2021,%_percentage_08/07/2021,price
0,0,0,BTC,2.588463e+06,2.458279e+06,-130183.900000,-5.029390,"₽2,450,745.58"
1,1,1,ETH,1.771903e+05,1.618009e+05,-15389.400000,-8.685239,"₽156,515.19"
2,2,2,USDT,7.445000e+01,7.502000e+01,0.570000,0.765615,₽74.32
3,3,3,BNB,2.516627e+04,2.354353e+04,-1622.740000,-6.448075,"₽23,036.35"
4,4,4,ADA,1.065400e+02,1.022300e+02,-4.310000,-4.045429,₽99.00
...,...,...,...,...,...,...,...,...
353,353,353,XTZDOWN,1.228100e-01,1.492900e-01,0.026480,21.561762,₽0.150147
354,354,354,XTZUP,3.350000e+00,2.630000e+00,-0.720000,-21.492537,₽2.60
355,355,355,YFIDOWN,1.471300e+02,1.883200e+02,41.190000,27.995650,₽196.47
356,356,356,YFIUP,4.518000e+01,3.346000e+01,-11.720000,-25.940682,₽31.37


In [15]:
import datetime as dt
from datetime import timedelta

In [16]:
today_date = 'dinamic', dt.datetime.today().strftime("%d/%m/%Y")
today_date = ''.join(today_date)
today_date = today_date[:7] + '_' + today_date[7:]
today_date

'dinamic_09/07/2021'

In [17]:
today_date_percentage = '%_percentage', dt.datetime.today().strftime("%d/%m/%Y")
today_date_percentage = ''.join(today_date_percentage)
today_date_percentage = today_date_percentage[:12] + '_' + today_date_percentage[12:]
today_date_percentage

'%_percentage_09/07/2021'

In [18]:
yesterday = dt.datetime.today() - timedelta(1)
yesterday_date_price_x = 'price', yesterday.strftime("%d/%m/%Y")
yesterday_date_price_x = ''.join(yesterday_date_price_x)
yesterday_date_price_x = yesterday_date_price_x[:5] + '_' + yesterday_date_price_x[5:]
yesterday_date_price_x

'price_08/07/2021'

In [19]:
today_date_price_y = 'price', dt.datetime.today().strftime("%d/%m/%Y")
today_date_price_y = ''.join(today_date_price_y)
today_date_price_y = today_date_price_y[:5] + '_' + today_date_price_y[5:]
today_date_price_y

'price_09/07/2021'

In [22]:
crypto_previous_day.price = crypto_previous_day.price.replace('₽', '', regex=True).replace(',', '', regex=True).astype('float')

In [33]:
d2 = {str(yesterday_date_price_x):'price_x', 'price':'price_y'}
crypto_previous_day.rename(columns=d2, inplace=True)

In [37]:
crypto_previous_day['today_date_crypto'] = (crypto_previous_day.price_y - crypto_previous_day.price_x)
crypto_previous_day['today_date_crypto_percentage'] = crypto_previous_day.today_date_crypto / \
                                                            crypto_previous_day.price_x * 100
d = {'today_date_crypto':today_date, 'today_date_crypto_percentage':today_date_percentage, 'price_x':yesterday_date_price_x, \
     'price_y':today_date_price_y}
crypto_previous_day.rename(columns=d, inplace=True)

In [43]:
import os.path

directory = 'C:/Users/under/Desktop/crypto_excels/'
file = 'crypto.xlsx'
crypto_previous_day.to_excel(os.path.join(directory, file))